####   D (англ. delete) — удалить,
####   I (англ. insert) — вставить,
####   R (replace) — заменить,
####   M (match) — совпадение.

####   w(a, а) = 0
####   w(a, b) = по формуле при a≠b - цена замены символа a на символ b
####   w(ε, b) = 1 - цена вставки символа b
####   w(a, ε) = 1 - цена удаления символа a

In [3]:
import numpy as np
import pandas as pd
import itertools

# the Levenshtein distance

Таблица всех звуков, преобразую ее.

In [14]:
data = pd.read_csv('/Users/Stoneberry/Desktop/курсач/3 курс/fon_table.csv/Лист 1-Tаблица 1.csv')
data = data.set_index('name')
data = data.transpose()
data = data.fillna('None')
data = pd.DataFrame(data)

Создаю будущую матрицу расстояний.

In [15]:
dist_matrix = pd.DataFrame(np.zeros((54, 54)))#, dtype=int))

In [16]:
column_names = {}
for i in range(0, 54):
    column_names[i] = data.columns.tolist()[i]

In [28]:
dist_matrix = dist_matrix.rename(index=str, columns=column_names)
dist_matrix['index'] = data.columns.tolist()
dist_matrix = dist_matrix.set_index('index')

Заполняю клеточки в таблице результатами формулы

1 - S_rows/ C_rows + Unc_rows*2 

In [30]:
comobinations = itertools.combinations_with_replacement(data.columns, 2) # - все возможные комбинации

In [31]:
for i in comobinations:

    a1 = pd.get_dummies(data[i[0]]).drop(['None'], axis=1)
    a = a1.loc[~(a1==0).all(axis=1)]
    b1 = pd.get_dummies(data[i[1]]).drop(['None'], axis=1)
    b = b1.loc[~(b1==0).all(axis=1)]
    
    dif_a = a.index.difference(b.index)
    similar = a.index.difference(dif_a)
    
    d = a.loc[similar] + b.loc[similar]
    s_rows = 0
    counts = d['+'].value_counts()

    if 0 in counts:
        s_rows += counts[0]
    if 2 in counts:
        s_rows += counts[2]
    
                        # common                uncommon
    result = 1 - s_rows/(len(similar) + (a.shape[0] + b.shape[0] - 2*len(similar)))# * 2)

    dist_matrix[i[0]][i[1]] = result
    dist_matrix[i[1]][i[0]] = result


Итог

In [33]:
dist_matrix.head()

,и,ы,е,э,ä,а,у,у_1,о,ъ,...,р’,j,к,к’,г,г’,х,х’,ɣ,ŋ
index,,,,,,,,,,,,,,,,,,,,,
и,0.000,0.125,0.125,0.250,0.250,0.375,0.250,0.125,0.375,0.375,...,0.785714,0.846154,1.000000,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571
ы,0.125,0.000,0.250,0.125,0.375,0.250,0.125,0.250,0.250,0.250,...,0.857143,0.923077,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571,0.857143
е,0.125,0.250,0.000,0.125,0.125,0.250,0.375,0.250,0.250,0.250,...,0.785714,0.846154,1.000000,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571
э,0.250,0.125,0.125,0.000,0.250,0.125,0.250,0.375,0.125,0.125,...,0.857143,0.923077,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571,0.857143
ä,0.250,0.375,0.125,0.250,0.000,0.125,0.500,0.375,0.375,0.375,...,0.785714,0.846154,1.000000,0.933333,1.000000,0.933333,1.000000,0.928571,1.000000,0.928571


##### Расстояние Левенштейна.


Я разобралась с визуализацией расстояния (http://www.let.rug.nl/kleiweg/lev/) и на осонове этой идеи сделала свою фунцию: 

##### Новая версия

In [169]:
def lev_distance(a, b, dist_matrix):
    
    # Первыми - строчки 
    # столбики - слово b
    
    dis = np.zeros((len(a) + 1, len(b) + 1))
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row - 1, col] + 1
        
  
        elif row > 1 and col > 1 and a[row-1] == b[col-2] and a[row-2] == b[col-1]:
            dis[row, col] = dis[row - 3][col - 3] + 1
            
        
        else:
            dis[row, col] = np.min([dis[row, col - 1] + 1,  # левый 
                                    dis[row - 1, col - 1] + dist_matrix[a[row-1]][b[col-1]], # диаг               
                                    dis[row - 1, col] + 1]) # верхний
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1

    print(dis)

    return dis[len(a), len(b)]

In [172]:
lev_distance('мила', 'мил', dist_matrix)

[[0. 1. 2. 3.]
 [1. 0. 1. 2.]
 [2. 1. 0. 1.]
 [3. 2. 1. 0.]
 [4. 3. 2. 1.]]


1.0

In [171]:
lev_distance('порт', 'спирт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 1.375      2.375      3.375     ]
 [3.         2.27272727 1.72727273 2.06320346 1.375      2.375     ]
 [4.         3.18181818 2.36363636 2.72727273 2.375      1.375     ]]


1.375

In [154]:
lev_distance('порт', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 1.         2.         3.        ]
 [3.         2.27272727 1.72727273 1.99177489 1.         2.        ]
 [4.         3.18181818 2.36363636 2.66060606 2.         1.        ]]


1.0

In [155]:
lev_distance('прот', 'порт', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.78571429 1.         2.        ]
 [3.         2.         1.         1.         1.93333333]
 [4.         3.         2.         1.36363636 1.        ]]


1.0

In [156]:
lev_distance('прот', 'пирт', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.85714286 1.         2.        ]
 [3.         2.         1.375      1.64285714 1.93333333]
 [4.         3.         2.375      1.73863636 1.64285714]]


1.6428571428571428

In [157]:
lev_distance('прот', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 0.72727273 1.72727273 2.         3.        ]
 [3.         2.27272727 1.72727273 0.72727273 2.         2.93333333]
 [4.         3.18181818 2.36363636 1.72727273 1.09090909 2.        ]]


2.0

In [173]:
lev_distance('мил', 'мила', dist_matrix)

[[0. 1. 2. 3. 4.]
 [1. 0. 1. 2. 3.]
 [2. 1. 0. 1. 2.]
 [3. 2. 1. 0. 1.]]


1.0

In [163]:
words = ['спорт', 'торт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word, dist_matrix)

{0.09090909090909094: ['торт'],
 0.25: ['парт'],
 1.0: ['спорт', 'прот'],
 1.2840909090909092: ['кипр'],
 1.375: ['спирт']}

In [164]:
words = ['мам', 'мамы',  'маму', 'мыла', 'папа', 'мышь', 'смама'] 
word = 'мама'
sort_words(words, word, dist_matrix)

{0.25: ['мамы'],
 0.375: ['маму'],
 0.5454545454545454: ['папа'],
 0.85: ['мыла'],
 1.0: ['смама'],
 1.2613636363636362: ['мышь'],
 2.0: ['мам']}

Проблема решена.

Здесь возникает некоторая проблема с перестановками. Так как этот алгорит сравнивает строчки линейно, то есть П и С, П и СП, П и СПО и тд. Моя идея заключалась в том, что замена происходит только тогда, когда буквы (сейчас я рассматриваю имеено буквы) находятся на одной и той же позиции, то есть П и С, ПО и СП и тд, то есть я сравниваю буквы именно тогда.
Здесь алгоритм не учитывает той ситуации, что две буквы могут совпадать и быть при этом не на одной и той же позиции, из-за этого получается так, что разница между спортом и портом 0.27, хотя должно быть 1. 

In [513]:
lev_distance1('порт', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.20606061 0.27272727 1.27272727 2.27272727]
 [3.         2.27272727 2.20606061 1.05844156 0.27272727 1.27272727]
 [4.         3.27272727 3.20606061 2.05844156 0.63636364 0.27272727]]


0.2727272727272727

In [520]:
lev_distance1('порт', 'спирт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.20606061 1.27272727 2.27272727 3.27272727]
 [3.         2.27272727 2.20606061 2.06320346 1.27272727 2.27272727]
 [4.         3.27272727 3.20606061 3.06320346 1.63636364 1.27272727]]


1.2727272727272727

Здесь я постаралась решить эту проблему, добавив условие, которое, в случае, когда буквы одинаковые но стоят на разных позициях в слове, ставит результатом их сравнения разницу в позициях. Если рассматривать случай порта и спорта, это равно 1, то есть, чтобы превраить СП в П, нужно удалить одну букву. 

In [516]:
def lev_distance2(a, b, dist_matrix):
    
    # Первыми - строчки 
    # столбики - слово b
    
    dis = np.zeros((len(a) + 1, len(b) + 1))
    i = 0
    row = 0
    col = 0

    while i < dis.size:

        if row == 0:
            if col != 0:
                dis[row, col] = dis[row, col-1] + 1

        elif col == 0:
            if row != 0:
                dis[row, col] = dis[row-1, col] + 1
        
        else:
   
            dis[row, col] = np.min([dis[row, col - 1],  # левый 
                                    dis[row - 1, col - 1], # диаг               
                                    dis[row - 1, col]]) # верхний
    
            if row == col:
                dis[row, col] += dist_matrix[a[row-1]][b[col-1]] # замена
                
            elif a[row-1] == b[col-1] and row != col:
                dis[row, col] = abs(row - col)  # совпадают на разных позициях
               
            elif a[row-1] != b[col-1]:
                dis[row, col] += 1 # все остальное
            
            #if dis[row, col] == dis[row, col-1]:
               # dis[row, col] = 1
               # dis[row, col-1] = 1
        
        col += 1
        i += 1  
        if col == len(b) + 1:
            col = 0
            row += 1
    print(dis)
    return dis[len(a), len(b)]

In [517]:
lev_distance2('порт', 'спорт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 1.         2.         3.        ]
 [3.         2.27272727 2.20606061 1.78571429 1.         2.        ]
 [4.         3.27272727 3.20606061 2.78571429 1.36363636 1.        ]]


1.0

Однако из-за этого теряется противопоставление и - о, которое хорошо разграничивалось в предыдущей формуле.

In [519]:
lev_distance2('порт', 'спирт', dist_matrix)

[[0.         1.         2.         3.         4.         5.        ]
 [1.         0.27272727 1.         2.         3.         4.        ]
 [2.         1.27272727 1.20606061 2.         3.         4.        ]
 [3.         2.27272727 2.20606061 2.06320346 1.         2.        ]
 [4.         3.27272727 3.20606061 3.06320346 1.36363636 1.        ]]


1.0

Тогда я решила попробовать внедрить способ, который использовала в предыдущих версиях - сравнивать линейные комбинации букв слова из словаря со словом, заданным пользователем. То есть я рассмотриваю комбинации спор, порт. Затем выбираю наименьшее расстояние до порта и прибавляю разницу в длине (= кол-во удалений). Расстояние я здесь считаю по lev_distance1

In [44]:
def sort_words(words, word, dist_matrix):
    
    result = {}
    
    for item in words:
        
        if len(item) != len(word):
            c = 0
            if len(item) < len(word):
                item, word = word, item
                c += 1

            diff = len(item) - len(word)
            l = len(word)
            var = len(item) - len(word)
    
            for i in range(diff + 1):
    
                dis = lev_distance(word, item[i:l], dist_matrix)
                if dis < var:
                    var = dis   
                l += 1
        
            dist = dis + diff
             
            if c != 0:
                item, word = word, item
        
        else:
            dist = lev_distance(word, item, dist_matrix)
        
        if dist not in result:
            result[dist] = [item]
        else:
            result[dist].append(item)

    return result

In [522]:
words = ['спирт']
word = 'порт'
sort_words(words, word, dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         1.         1.         2.         3.        ]
 [1.         1.         1.93333333 2.         3.        ]
 [1.         1.         2.         2.79047619 2.        ]
 [1.         1.         2.         3.         2.36363636]]
[[0.    1.    2.    3.    4.   ]
 [1.    0.    1.    2.    3.   ]
 [2.    1.    0.375 1.375 2.375]
 [3.    2.    1.375 0.375 1.375]
 [4.    3.    2.375 1.375 0.375]]


{1.375: ['спирт']}

In [523]:
words = ['спорт']
word = 'порт'
sort_words(words, word, dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         1.         1.         2.         3.        ]
 [1.         1.         1.93333333 1.         2.        ]
 [1.         1.         2.         1.78571429 1.        ]
 [1.         1.         2.         2.78571429 1.36363636]]
[[0. 1. 2. 3. 4.]
 [1. 0. 1. 2. 3.]
 [2. 1. 0. 1. 2.]
 [3. 2. 1. 0. 1.]
 [4. 3. 2. 1. 0.]]


{1.0: ['спорт']}

In [495]:
words = ['спорт', 'торт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word, dist_matrix)

{0.09090909090909094: ['торт'],
 0.25: ['парт'],
 1.0: ['спорт'],
 1.2840909090909092: ['кипр'],
 1.375: ['спирт'],
 1.5714285714285714: ['прот']}

In [496]:
words = ['спорт', 'прт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word, dist_matrix)

{0.25: ['парт'],
 1.0: ['спорт'],
 1.2840909090909092: ['кипр'],
 1.375: ['спирт'],
 1.5714285714285714: ['прот'],
 1.9333333333333333: ['прт']}

In [497]:
words = ['мам', 'мамы',  'маму', 'мыла', 'папа', 'мышь', 'смама'] 
word = 'мама'
sort_words(words, word, dist_matrix)

{0.25: ['мамы'],
 0.375: ['маму'],
 0.5454545454545454: ['папа'],
 0.85: ['мыла'],
 1.0: ['смама'],
 1.2613636363636362: ['мышь'],
 2.7142857142857144: ['мам']}

С этой точки зрения проблема решена, однако есть переставноки вида порт и прот. 

In [531]:
lev_distance2('порт', 'прот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.78571429 1.         2.        ]
 [3.         2.         1.         1.57142857 2.        ]
 [4.         3.         2.         2.         1.57142857]]


1.5714285714285714

In [532]:
lev_distance1('порт', 'прот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.         1.         2.         3.        ]
 [2.         1.         0.78571429 0.78571429 1.78571429]
 [3.         2.         0.78571429 1.57142857 1.78571429]
 [4.         3.         1.78571429 1.78571429 1.57142857]]


1.5714285714285714

Самое простое решение, которое я вижу - рассматривать расстояние вообще ото всех возможных комбинаций длины заданного слова, однако это сулит большим временем работы.

Сейчас я пытаюсь, что-то придумать пути с изменение результатов в матрице. Например, если у нас есть последовательность из двух одинаковых чисел, то, возможно, в этом месте есть перестановка, однако пока не очень понятно, как сохранить разницу между предшедмтвующими звуками

In [534]:
lev_distance2('порт', 'мрот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.05844156 1.         2.        ]
 [3.         2.27272727 1.         1.78571429 2.        ]
 [4.         3.27272727 2.         2.         1.78571429]]


1.7857142857142856

In [535]:
lev_distance1('порт', 'мрот', dist_matrix)

[[0.         1.         2.         3.         4.        ]
 [1.         0.27272727 1.27272727 2.27272727 3.27272727]
 [2.         1.27272727 1.05844156 1.05844156 2.05844156]
 [3.         2.27272727 1.05844156 1.84415584 2.05844156]
 [4.         3.27272727 2.05844156 2.05844156 1.84415584]]


1.844155844155844

#### Фонетические изменения

In [281]:
              #    v2   v1   V   vn
vows = {'а': ['а', 'ъ', 'а', 'а', 'ъ'],
        'я': ['а', 'ь', 'и', 'а', 'ь'],
        'о': ['а', 'ъ', 'а', 'о', 'ъ'],
        'ё': ['а', 'ь', 'и', 'о', 'ь'],
        'у': ['у', 'у', 'у', 'у', 'у'], 
        'ю': ['у', 'у', 'у', 'у', 'у'],
        'е': ['и', 'ь', 'и', 'е', 'ь'],  
        'э': ['и', 'ъ', 'ы', 'э', 'ъ'], 
        'и': ['и', 'ь', 'и', 'и', 'ь'],
        'ы': ['и', 'ъ', 'ы', 'ы', 'ъ'] 
        }

In [19]:
def levenshtein(s, t):
        ''' From Wikipedia article; Iterative with two matrix rows. '''
        if s == t: return 0
        elif len(s) == 0: return len(t)
        elif len(t) == 0: return len(s)
        v0 = [None] * (len(t) + 1)
        v1 = [None] * (len(t) + 1)
        for i in range(len(v0)):
            v0[i] = i
        for i in range(len(s)):
            v1[0] = i + 1
            for j in range(len(t)):
                cost = 0 if s[i] == t[j] else 1
                v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
            for j in range(len(v0)):
                v0[j] = v1[j]
                
        return v1[len(t)]

In [20]:
levenshtein('post', 'psot')

2

## СТАРЫЕ НАРАБОТКИ

In [115]:
def dd(a, b):

    dis = np.zeros((len(a) + 1, len(b) + 1))
    
    mistakes = [0]*len(a)
    row = 0
    col = 0
    C = 1
    i = 0
    answer = 0

    while i < dis.size:
    
        if row != len(a):
            if a[row] == b[col]:
                if C == 0:
                    continue
        
                if row != col:
                    mistakes[row] += 1
                dis[row, col] += 1
                C -= 1
        
      #  if row == col:
          #  if a[row] != b[col]
            
     
            if col == len(b) - 1:
                dis[row, col+1] = sum(dis[row])
                col = 0
                row += 1
                C = 1
        
        else:
            dis[row, col-1] = sum(dis[col-1][0:len(a)])
            if col-1 == len(b):
                break
        col += 1

        i+= 1
    print(dis)
    print(mistakes)
    return answer

Датафрейм с пропущенными значениями

In [20]:
data = data.transpose()

In [24]:
data.isnull().sum()['и']

7

In [28]:
distance(data['ä'], data['и'])

9

In [110]:
data['и'].values

array([ 1.,  1.,  0., nan, nan,  1.,  0.,  0.,  0.,  1., nan, nan, nan,
       nan, nan])

In [111]:
data['к'].values

array([ 0.,  0.,  1.,  0.,  0., nan, nan,  1., nan, nan,  0.,  0.,  0.,
        0.,  0.])

Функция считает NAn как различие

#### Евклидово расстрояние между строками. 

Датафрейм 1 = 2, 0 = 1, Nan = 0

In [173]:
from scipy.spatial.distance import pdist, squareform

data2 = data2.fillna(0)
distances = pdist(data2.values, metric='euclidean')
dist_matrix = squareform(distances)
dist_matrix = pd.DataFrame(dist_matrix)
dist_matrix = dist_matrix.rename(index=str, columns=column_names)
dist_matrix['index'] = data2.index.tolist()
dist_matrix = dist_matrix.set_index('index')

0 - одинаковы

до 2 - похожи

выше 4.582576 - уже точно не похожи

In [174]:
dist_matrix

,и,ы,е,э,ä,а,у,у_1,о,ъ,...,р’,j,к,к’,г,г’,х,х’,ɣ,ŋ
index,,,,,,,,,,,,,,,,,,,,,
и,0.000000,1.000000,1.000000,1.414214,1.414214,1.732051,1.414214,1.000000,1.732051,1.732051,...,4.472136,4.472136,4.582576,4.472136,4.898979,4.795832,4.795832,4.690416,5.099020,4.690416
ы,1.000000,0.000000,1.414214,1.000000,1.732051,1.414214,1.000000,1.414214,1.414214,1.414214,...,4.582576,4.582576,4.472136,4.582576,4.795832,4.898979,4.690416,4.795832,5.000000,4.582576
е,1.000000,1.414214,0.000000,1.000000,1.000000,1.414214,1.732051,1.414214,1.414214,1.414214,...,4.123106,4.123106,4.242641,4.123106,4.582576,4.472136,4.472136,4.358899,4.795832,4.358899
э,1.414214,1.000000,1.000000,0.000000,1.414214,1.000000,1.414214,1.732051,1.000000,1.000000,...,4.242641,4.242641,4.123106,4.242641,4.472136,4.582576,4.358899,4.472136,4.690416,4.242641
ä,1.414214,1.732051,1.000000,1.414214,0.000000,1.000000,2.000000,1.732051,1.732051,1.732051,...,4.472136,4.472136,4.582576,4.472136,4.898979,4.795832,4.795832,4.690416,5.099020,4.690416
а,1.732051,1.414214,1.414214,1.000000,1.000000,0.000000,1.732051,2.000000,1.414214,1.414214,...,4.582576,4.582576,4.472136,4.582576,4.795832,4.898979,4.690416,4.795832,5.000000,4.582576
у,1.414214,1.000000,1.732051,1.414214,2.000000,1.732051,0.000000,1.000000,1.000000,1.732051,...,4.898979,4.898979,4.795832,4.898979,5.099020,5.196152,5.000000,5.099020,5.291503,4.898979
у_1,1.000000,1.414214,1.414214,1.732051,1.732051,2.000000,1.000000,0.000000,1.414214,2.000000,...,4.795832,4.795832,4.898979,4.795832,5.196152,5.099020,5.099020,5.000000,5.385165,5.000000
о,1.732051,1.414214,1.414214,1.000000,1.732051,1.414214,1.000000,1.414214,0.000000,1.414214,...,4.582576,4.582576,4.472136,4.582576,4.795832,4.898979,4.690416,4.795832,5.000000,4.582576


In [180]:
def mydistance(word1, word2):

    word1 = list(word1)
    word2 = list(word2)
    answer = []
    ans = 0
    for index, item in enumerate(word1):
        answer.append(dist_matrix[item][word2[index]])
        ans += dist_matrix[item][word2[index]]
    return answer, ans

In [132]:
mydistance('порт', 'торт')[1]

1.0

In [133]:
mydistance('порт', 'парт')[1]

1.4142135623730951

In [134]:
mydistance('порт', 'прот')[1]

8.94427190999916

In [135]:
mydistance('порт', 'спорт')[1]

13.000018285881175

In [139]:
mydistance('порт', 'спирт')[1]

13.110458025837435

In [209]:
mydistance('порт', 'кипр')[1]*4

27.872475140274666

In [136]:
distance('порт', 'спорт')

1

In [137]:
distance('порт', 'парт')

1

In [141]:
distance('порт', 'кипр')

4

In [272]:
def sort_words(words, word):
    answer = {}
    for item in words:

        distance = lev_distance(word, item)

        if distance == 0:
            continue

        var = mydistance(word, item)[1]
        
        if len(word) < len(item):

            diff = len(item) - len(word)
            l = len(word)
            
            for i in range(diff + 1):
                dis = mydistance(word, item[i:l])[1]
                if dis < var:
                    var = dis
                    l += 1

            var += diff

      #  if mydistance(word, answer[distance][-2])[1] > var:
         #   answer[distance][-1], answer[distance][-2] = answer[distance][-2], answer[distance][-1]
    
        if distance*var in answer:
            answer[distance*var].append(item)
        else:
            answer[distance*var] = [item]
    return answer

In [273]:
words = ['спорт', 'торт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word)

{1.0: ['спорт', 'торт'],
 1.4142135623730951: ['парт'],
 5.464101615137754: ['спирт'],
 17.88854381999832: ['прот'],
 27.872475140274666: ['кипр']}

In [274]:
words = ['спорт', 'прт', 'прот', 'парт', 'спирт', 'кипр'] 
word = 'порт'
sort_words(words, word)

{1.0: ['спорт'],
 1.4142135623730951: ['парт'],
 5.464101615137754: ['спирт'],
 6.47213595499958: ['прт'],
 17.88854381999832: ['прот'],
 27.872475140274666: ['кипр']}

In [275]:
words = ['мам', 'мамы',  'маму', 'мыла', 'папа', 'мышь', 'смама'] 
word = 'мама'
sort_words(words, word)

{0.0: ['мам'],
 1.0: ['смама'],
 1.4142135623730951: ['мамы'],
 1.7320508075688772: ['маму'],
 6.928203230275509: ['папа'],
 7.727406610312546: ['мыла'],
 17.37604704301969: ['мышь']}

Пробую решить проблемы спорта и порта - передвижение

In [204]:
word1 = 'порт'
word2 = 'транспорт'
variant = 0
if len(word1) != len(word2):
    
    dfrnc = len(word2) - len(word1)
    l = len(word1)
    
    for i in range(dfrnc+1):

        if mydistance(word1, word2[i:l])[1] < variant:
            variant = mydistance(word2[i:l])
        l += 1
variant

0

Пробую решить проблемы порта и прта - вставка

Пробую решить проблемы порта и портам - удаление

In [277]:
lev_distance('порт', 'прт')

1

In [266]:
mydistance('порт', 'прт')[0]

[0.0, 4.47213595499958, 2.0]

In [ ]:
def sort_words2(words, word):
    answer = {}
    for item in words:

        distance = lev_distance(word, item)
        var = mydistance(word, item)[1]
        
        if len(word) < len(item):
            

            diff = len(item) - len(word)
            l = len(word1)
            
            for i in range(diff + 1):
                dis = mydistance(word, item[i:l])[1]
                if dis < var:
                    var = dis
                    l += 1
    
      #  if mydistance(word, answer[distance][-2])[1] > var:
         #   answer[distance][-1], answer[distance][-2] = answer[distance][-2], answer[distance][-1]
        if distance*var in answer:
            answer[distance*var].append(item)
        else:
            answer[distance*var] = [item]
    return answer

#### Попытка посчитать расстояние Levenshtein distance

In [ ]:
result = data.apply(lambda col1: data.apply(lambda col2: distance(col1, col2)))- data.isnull().sum()['и']
result

In [ ]:
cons = {'б': ['п', ], 'г': ['к', 'г’'], 
        'к': ['к'], 'д': ['т'], 
        'т': ['т'], 'з': ['с'], 
        'с': ['с'], 'ж': ['ш'], 
        'п': ['п'], 'й': ['й'], 
        'м': ['м'], 'л': ['л'], 
        'р': ['р'], 'в': ['ф'], 
        'ф': ['ф'], 'х': ['х'], 
        'щ': ['щ'],'ш': ['ш'], 
        'ц': ['ц'], 'нг': ['ŋ']}

In [ ]:
if word[-1]

In [17]:
from scipy.spatial.distance import pdist, squareform

distances = pdist(data)
dist_matrix = squareform(distances)

In [19]:
pd.DataFrame(dist_matrix)

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Загружаю признаковую таблицу

In [17]:
x = data.drop('name', 1)
Y = data['name'] 
data

,name,son,voc,cons,cor,ant,high,low,back,md,tns,nsl,ltr,cont,drls,vcd
0,и,+,+,-,0,0,+,-,-,-,+,0,0,0,0,0
1,ы,+,+,-,0,0,+,-,+,-,+,0,0,0,0,0
2,е,+,+,-,0,0,-,-,-,-,+,0,0,0,0,0
3,э,+,+,-,0,0,-,-,+,-,+,0,0,0,0,0
4,ä,+,+,-,0,0,-,+,-,-,+,0,0,0,0,0
5,а,+,+,-,0,0,-,+,+,-,+,0,0,0,0,0
6,у,+,+,-,0,0,+,-,+,+,+,0,0,0,0,0
7,у,+,+,-,0,0,+,-,-,+,+,0,0,0,0,0
8,о,+,+,-,0,0,-,-,+,+,+,0,0,0,0,0
9,ъ,+,+,-,0,0,-,-,+,-,-,0,0,0,0,0


Тут я делаю словарь с ключом - звук, значение - его индекс, для более простого обращения к индексам

In [6]:
y = {} 

for index, item in enumerate(Y):
    y[item] = index

Расстояние между "мам" и "мама"

In [7]:
m1 = list('мам')
m2 = list('мама')

In [8]:
def searching_f(letters, y, x):
    word = []
    # на вход массив букв
    for letter in letters:
        word.append(list(x.ix[y[letter]]))
    return word
     

In [9]:
mam = searching_f(m1, y, x)
mama = searching_f(m2, y, x)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [10]:
distance(mam, mama)

1

In [11]:
m3 = list('мим')
mim = searching_f(m3, y, x)
distance(mam, mim)

1

In [12]:
distance(mim, mama)

2

In [14]:
k1 = list('кирстф')
kirst = searching_f(k1, y, x)

v1 = list('вампир')
vampir = searching_f(v1, y, x)
distance(kirst, vampir)

6

In [16]:
k2 = list('карстф')
karst = searching_f(k2, y, x)

distance(karst, vampir)

5

Матрицу рас. между звуками и звуками

0 заменить None

In [ ]:
new_data